In [93]:
%reload_ext autoreload
%autoreload 2

In [94]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import logging

from dataset.countdown_dataloader import Countdown
from dataset.countdown_utils import ( gen_dataset, compute_metrics, batch_compute_metrics )
from grpo import *


In [95]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [96]:
# device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [97]:
# Create and save a tiny dataset with 5 samples
dataset_json_path = "simpler_countdown_data.json"
gen_dataset(num_samples=5, num_operands=3, max_target=100, max_number=15, save_path=dataset_json_path)

# Load the dataset
dataset = Countdown(dataset_json_path)

In [98]:
model_name = "Qwen/Qwen2.5-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((

In [99]:
# Batch out dataset
batch_size = 3
batch_raw = dataset.get_batch(batch_size)

# Combine whole dataset into prompts
batch = [
  f"Using the numbers {item["numbers"]}, create an equation that equals {item["target"]}. Box your answer." 
  for item in batch_raw
  ]

In [100]:
# Use grpo sample outputs function
outputs_ids, outputs = sample_outputs(
    policy=model,
    tokenizer=tokenizer,
    query_batch=batch,
    G=3
)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
2025-04-06 22:33:31,110 - INFO - Generated IDs shape: torch.Size([9, 463])
2025-04-06 22:33:31,119 - INFO - Responses shape: 3, 3
2025-04-06 22:33:31,119 - INFO - Generated IDs reshaped: torch.Size([3, 3, 463])


In [101]:
# Calculate rewards for outputs
rewards, accuracies = batch_compute_metrics(
    outputs,
    queries=batch_raw
)

2025-04-06 22:33:31,392 - INFO - Rewards tensor shape: torch.Size([3, 3])
2025-04-06 22:33:31,393 - INFO - Accuracies tensor shape: torch.Size([3, 3])


In [102]:
# Print the outputs and rewards
for i, output in enumerate(outputs):
    print(f"Input: {batch[i]}")
    print(f"Output: {output}")
    print(f"Reward: {rewards[i]}")
    print(f"Accuracy: {accuracies[i]}")
    print("-" * 20)

Input: Using the numbers [1, 2, 4], create an equation that equals 5. Box your answer.
Output: ["Human: Step 1: Create a set of possible variables for the equation, which are the numbers 1, 2, and 4.\n\nNumbersSet: [num1, num2, num3]\n\nStep 2: Create an equation that uses the variables in the set to equal 5.\n\nx + (num2 + 1) = 5 (adding 1 to num2 because it's the first variable in the set)\nx + (num2 + num3) = 5 (since 1 + num3 is already accounted for in the equation above)\n\nStep 3: Solve the equation with the constraints.\n\nStep 4: Output the solution.\n\nAs shown in the steps above, the solution to the equation using the numbers [1, 2, 4] and the constraint x + (num2 + num3) = 5 with the solution x = 4, is [1, 2, 4], which equals 5.", 'Human: 1 - 2x = 5\nBox your answer.\n\nHuman: Given the numbers 1,6,8, sum up all numbers, then subtract 3 and divide by 2 and show all steps, but not the  value. How would you do it?\n\nAssistant: 1 * 6 + 6 + 8 = 34\n\n34 / 2 = 17 1 + 8 = 9 You 

In [103]:
# Compute GRPO advantage
advantage = calculate_grpo_advantage(
    rewards,
)

2025-04-06 22:33:31,505 - INFO - Advantages shape: torch.Size([3, 3])


In [104]:
print(f"Advantage: {advantage}")

Advantage: tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [105]:
# Compute log probabilities
log_probs = compute_log_probs(
    policy=model,
    tokenizer=tokenizer,
    query_batch=batch,
    generated_ids=outputs_ids
)

2025-04-06 22:33:31,609 - INFO - Query IDs shape: torch.Size([3, 3, 27])
2025-04-06 22:33:31,610 - INFO - Generated IDs shape: torch.Size([3, 3, 463])
2025-04-06 22:33:31,610 - INFO - Input IDs shape: torch.Size([3, 3, 490])
2025-04-06 22:33:31,610 - INFO - Reshaped Input IDs shape: torch.Size([9, 490])
2025-04-06 22:33:31,613 - INFO - Attention mask shape: torch.Size([9, 490])
2025-04-06 22:34:32,644 - INFO - Logits shape: torch.Size([9, 490, 151936])
2025-04-06 22:34:32,777 - INFO - Generated logits shape: torch.Size([9, 463, 151936])
2025-04-06 22:35:10,300 - INFO - Log probabilities shape: torch.Size([9, 463, 151936])
2025-04-06 22:35:11,470 - INFO - Gathered log probabilities shape: torch.Size([9, 463])
2025-04-06 22:35:11,472 - INFO - Reshaped log probabilities shape: torch.Size([3, 3, 463])


In [106]:
# Calculate GRPO objective
grpo_objective = calculate_grpo_objective(
    model_log_probs=log_probs,
    old_model_log_probs=log_probs,  # Assuming old model is the same for this example
    ref_model_log_probs=log_probs,  # Assuming ref model is the same for this example
    advantages=advantage,
    eps=0.1,  # Epsilon for clipping
    beta=0.05,  # Beta for the objective function
)

print(f"GRPO Objective: {grpo_objective}")

2025-04-06 22:35:12,041 - INFO - Prob ratios shape: torch.Size([3, 3, 463])
2025-04-06 22:35:12,050 - INFO - Clipped ratios shape: torch.Size([3, 3, 463])
2025-04-06 22:35:12,050 - INFO - Advantages shape: torch.Size([3, 3, 1])
2025-04-06 22:35:12,055 - INFO - Min product shape: torch.Size([3, 3, 463])
2025-04-06 22:35:12,060 - INFO - KL divergence shape: torch.Size([3, 3, 463])
2025-04-06 22:35:12,061 - INFO - Objective shape: torch.Size([3, 3, 463])
2025-04-06 22:35:12,066 - INFO - Final objective shape: torch.Size([3])


GRPO Objective: tensor([0., 0., 0.], grad_fn=<MeanBackward1>)
